In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime


In [25]:
df = pd.read_csv('BTC-price-history(1).csv')

,Unnamed: 0,Date,Price
0,0,2012-12-31,13.44
1,1,2013-01-01,13.31
2,2,2013-01-02,13.29
3,3,2013-01-03,13.32
4,4,2013-01-04,13.38
...,...,...,...
3960,3960,2023-11-04,35101.00
3961,3961,2023-11-05,35038.00
3962,3962,2023-11-06,34997.00
3963,3963,2023-11-07,35440.00


In [ ]:
def fill_missing_prices(csv_path, symbol):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Convert the 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Get the last date in the CSV
    last_date = df['Date'].max()
    
    # Download historical data from Yahoo Finance for the given symbol from the last date to today
    yahoo_data = yf.download(symbol, start=last_date, end=pd.to_datetime('today').strftime('%Y-%m-%d'))
    
    # Merge the downloaded data with the existing data based on the 'Date' column
    merged_data = pd.merge(df, yahoo_data['Adj Close'], how='outer', left_on='Date', right_index=True)
    
    # Update the 'Adj Close' column with the downloaded prices where missing
    merged_data['Adj Close_x'].update(merged_data['Adj Close_y'])
    
    # Drop the redundant columns and rename the updated 'Adj Close' column
    merged_data = merged_data[['Date', 'Adj Close_x']].rename(columns={'Adj Close_x': 'Adj Close'})
    
    # Calculate 111 DMA and 350 DMA * 2 for the newly added days
    merged_data['111_DMA'] = merged_data['Adj Close'].rolling(window=111).mean()
    merged_data['350_DMA_2x'] = merged_data['Adj Close'].rolling(window=350).mean() * 2
    
    return merged_data


In [ ]:
csv_path = 'BTC-price-history(1).csv'
symbol = 'BTC-USD'

filled_data = fill_missing_prices(csv_path, symbol)
print(filled_data)

In [27]:
dma = btc_price['Price'].rolling(window=350).mean()
btc_price['350_DMA_2x'] = dma * 2

In [43]:
btc_price

,Unnamed: 0,Date,Price,111_DMA,350_DMA_2x,Crossover
0,0,2012-12-31,13.44,NaN,NaN,0
1,1,2013-01-01,13.31,NaN,NaN,0
2,2,2013-01-02,13.29,NaN,NaN,0
3,3,2013-01-03,13.32,NaN,NaN,0
4,4,2013-01-04,13.38,NaN,NaN,0
...,...,...,...,...,...,...
3960,3960,2023-11-04,35101.00,28459.657658,51231.764343,-1
3961,3961,2023-11-05,35038.00,28503.747748,51339.320457,-1
3962,3962,2023-11-06,34997.00,28550.054054,51448.851086,-1
3963,3963,2023-11-07,35440.00,28599.828829,51558.763486,-1


In [29]:
# Identify crossover points
btc_price['Crossover'] = 0
btc_price.loc[btc_price['111_DMA'] > btc_price['350_DMA_2x'], 'Crossover'] = 1
btc_price.loc[btc_price['111_DMA'] < btc_price['350_DMA_2x'], 'Crossover'] = -1

In [34]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=btc_price['Date'], y=btc_price['Price'], mode='lines', name='Bitcoin Price'))
fig.add_trace(go.Scatter(x=btc_price['Date'], y=btc_price['111_DMA'], mode='lines', name='111 DMA'))
fig.add_trace(go.Scatter(x=btc_price['Date'], y=btc_price['350_DMA_2x'], mode='lines', name='350 DMA * 2'))
fig.update_layout(
    title='Bitcoin Price with Moving Averages and Crossover Lines (Log Scale)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price', type='log'),
    legend=dict(x=0, y=1, traceorder='normal'),
)
fig.show()